In [18]:
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
!pip install kaggle

In [7]:
!kaggle competitions download -c ds3-predictive-modeling-challenge

 53%|███████████████████▉                  | 2.00M/3.81M [00:00<00:00, 16.3MB/s]
100%|██████████████████████████████████████| 3.81M/3.81M [00:00<00:00, 21.4MB/s]
  0%|                                                | 0.00/211k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 211k/211k [00:00<00:00, 58.0MB/s]
  0%|                                                | 0.00/948k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 948k/948k [00:00<00:00, 36.6MB/s]
  0%|                                                | 0.00/236k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 236k/236k [00:00<00:00, 55.3MB/s]


In [289]:
data         = pd.read_csv('train_features.csv')
feature_info = pd.read_csv('train_labels.csv')
X_val        = pd.read_csv('test_features.csv')
sample_submi = pd.read_csv('sample_submission.csv')

data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [290]:
X = data
y = feature_info['status_group']

## Establish a baseline and Test Submission

In [22]:
feature_info['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [23]:
baseline = feature_info.copy()

In [27]:
baseline['base_status'] = 'functional'
baseline_pred = baseline['base_status']
baseline_pred

In [35]:
accuracy_score(y, baseline_pred)

0.543080808080808

In [37]:
!kaggle competitions submit ds3-predictive-modeling-challenge -f sample_submission.csv -m "Baseline Test and Kaggle API submission test."

100%|████████████████████████████████████████| 236k/236k [00:03<00:00, 65.1kB/s]
Successfully submitted to DS3 Predictive Modeling Challenge

### Train Accuracy_score:
0.543080808080808

### Test Accuracy_score:
0.54157

The datasets seem to have a similar distribution based on this information. 

In [43]:
X.select_dtypes('number').isnull().sum()

id                   0
amount_tsh           0
gps_height           0
longitude            0
latitude             0
num_private          0
region_code          0
district_code        0
population           0
construction_year    0
dtype: int64

In [45]:
numeric = X.select_dtypes('number')

In [49]:
# Randomized search to find the optimal hyperparameters. 
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

param_distributions = {
    'n_estimators': randint(50, 500), 
    'max_depth': randint(1,5)
}

search = RandomizedSearchCV(
    estimator = XGBClassifier(n_jobs = -1, random_state = 42),
    param_distributions = param_distributions, 
    scoring = 'accuracy',
    n_iter = 10,
    n_jobs = -1,
    cv= 3,
    verbose = 10,
    return_train_score = True,
    random_state = 42
)

search.fit(numeric, y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:   57.1s remaining:   33.1s
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:  1.1min remaining:   19.5s
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:  1.3min remaining:    9.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.4min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x126c9b320>, 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a2b36f358>},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=True, scoring='accuracy', verbose=10)

In [50]:
# creating variable name for the best_estimator object. 
best = search.best_estimator_
# Best Accuracy Score
search.best_score_

0.6675420875420875

In [55]:
# Make a prediciton on the test data
X_val_numeric = X_val.select_dtypes('number')
y_pred = best.predict(X_val_numeric)

Exception ignored in: <function DMatrix.__del__ at 0x126d59d90>
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/xgboost/core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


In [61]:
#Numeric Baseline
sample_submi['status_group'] = y_pred
sample_submi.to_csv("numeric_only_model.csv" , index = False)

In [62]:
!kaggle competitions submit ds3-predictive-modeling-challenge -f numeric_only_model.csv -m "Numeric only model test. Determine variation in distribution between train and test"

100%|█████████████████████████████████████████| 256k/256k [00:00<00:00, 355kB/s]
Successfully submitted to DS3 Predictive Modeling Challenge

### Train Accuracy Score: 
0.6675420875420875

### Test Accuracy Score: 
0.67000

In [67]:
X_train, X_test, y_train, y_test = train_test_split(numeric, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((47520, 10), (11880, 10), (47520,), (11880,))

In [68]:
# Lets Cross Reference that with the Permutation Importance. 

import eli5
from eli5.sklearn import PermutationImportance

# In its current form, eli5 doesn't work with pandas and XGBoost. It needs to be an array. 
## Likely because it renames the columns to some hardcoded value. 
### https://www.kaggle.com/dansbecker/permutation-importance#392299
X_train_dbg = X_train.values
y_train_dbg = y_train.values
X_test_dbg = X_test.values
y_test_dbg = y_test.values

# The model also needs to fit with the nd.array data type
best.fit(X_train_dbg, y_train_dbg)

permuter = PermutationImportance(best, scoring = 'accuracy',
                                 n_iter = 4, random_state = 42)

permuter.fit(X_test_dbg, y_test_dbg)

feature_names = X_test.columns.tolist()
eli5.show_weights(permuter, top = None, feature_names = feature_names)

Exception ignored in: <function DMatrix.__del__ at 0x126d59d90>
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/xgboost/core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


Weight,Feature
0.0681 ± 0.0047,latitude
0.0568 ± 0.0034,amount_tsh
0.0528 ± 0.0030,construction_year
0.0488 ± 0.0034,longitude
0.0293 ± 0.0033,population
0.0151 ± 0.0023,region_code
0.0124 ± 0.0069,district_code
0.0123 ± 0.0054,gps_height
0.0008 ± 0.0005,num_private
0.0003 ± 0.0021,id


### Good Numeric Features. 

All the features appear to have some form of Permutation Importance. From here I am going impute, encode and go for a kitchen sink model. 

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((47520, 40), (11880, 40), (47520,), (11880,))

In [84]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
import category_encoders as ce

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer (strategy = 'most_frequent'),
    best
)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('ordinalencoder', OrdinalEncoder(cols=['date_recorded', 'funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'recorded_by', 'scheme_management', 'scheme_name', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'mana...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

In [85]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipeline, X_train, y_train, scoring='accuracy', cv=3).mean()

0.6999955950622371

In [86]:
y_pred = pipeline.predict(X_val)
sample_submi['status_group'] = y_pred
sample_submi.to_csv("Kitchen_Sink.csv" , index = False)

In [87]:
!kaggle competitions submit ds3-predictive-modeling-challenge -f Kitchen_sink.csv -m "Kitchen Sink with Ordinal Encoder and Mean Imputer"

100%|████████████████████████████████████████| 258k/258k [00:03<00:00, 69.0kB/s]
Successfully submitted to DS3 Predictive Modeling Challenge

### Train Accuracy Score: 
0.6999955950622371

### Test Accuracy Score: 
0.76681

In [100]:
pre_processor = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer (strategy = 'most_frequent')
)

X_train = pre_processor.fit_transform(X_train)
X_test = pre_processor.fit_transform(X_test)

In [102]:
best.fit(X_train, y_train)

permuter = PermutationImportance(best, scoring = 'accuracy',
                                 n_iter = 4, random_state = 42)

permuter.fit(X_test, y_test)

feature_names = X.columns.tolist()
eli5.show_weights(permuter, top = None, feature_names = feature_names)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [103]:
#feature_names = X.columns.tolist()
#eli5.show_weights(permuter, top = None, feature_names = feature_names)

Weight,Feature
0.0957 ± 0.0040,quantity
0.0234 ± 0.0025,waterpoint_type
0.0185 ± 0.0026,extraction_type_class
0.0145 ± 0.0017,longitude
0.0145 ± 0.0042,construction_year
0.0132 ± 0.0011,payment
0.0094 ± 0.0018,latitude
0.0088 ± 0.0020,lga
0.0081 ± 0.0035,amount_tsh
0.0079 ± 0.0027,extraction_type


In [ ]:
mask = permuter.feature_importances_ > 0
features = X.columns[mask]

In [113]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer (strategy = 'most_frequent'),
    best
)

pipeline.fit(X[features], y)

Pipeline(memory=None,
     steps=[('ordinalencoder', OrdinalEncoder(cols=['date_recorded', 'funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'scheme_management', 'scheme_name', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'manag...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

In [116]:
cross_val_score(pipeline, X[features], y, scoring='accuracy', cv=3).mean()

Exception ignored in: <function DMatrix.__del__ at 0x126d59d90>
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/xgboost/core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'
Exception ignored in: <function DMatrix.__del__ at 0x126d59d90>
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/xgboost/core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


0.7473063973063973

In [117]:
y_pred = pipeline.predict(X_val[features])
sample_submi['status_group'] = y_pred
sample_submi.to_csv("Kitchen_Sink_Permuated.csv" , index = False)

In [118]:
!kaggle competitions submit ds3-predictive-modeling-challenge -f Kitchen_Sink_Permuated.csv -m "Now with noisy features removed."

100%|█████████████████████████████████████████| 258k/258k [00:00<00:00, 314kB/s]
Successfully submitted to DS3 Predictive Modeling Challenge

### Train Accuracy Score: 
0.7473063973063973

### Test Accuracy Score: 
0.77475

In [122]:
from sklearn.preprocessing import MinMaxScaler

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer (strategy = 'most_frequent'),
    MinMaxScaler(),
    best
)

pipeline.fit(X[features], y)

Pipeline(memory=None,
     steps=[('ordinalencoder', OrdinalEncoder(cols=['date_recorded', 'funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'scheme_management', 'scheme_name', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'manag...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

In [123]:
cross_val_score(pipeline, X[features], y, scoring='accuracy', cv=3).mean()

0.7471380471380472

In [124]:
y_pred = pipeline.predict(X_val[features])
sample_submi['status_group'] = y_pred
sample_submi.to_csv("Kitchen_Sink_Permuted_Scaled.csv" , index = False)

In [125]:
!kaggle competitions submit ds3-predictive-modeling-challenge -f Kitchen_Sink_Permuted_Scaled.csv -m "Checking Scaling on for Size"

100%|█████████████████████████████████████████| 258k/258k [00:00<00:00, 321kB/s]
Successfully submitted to DS3 Predictive Modeling Challenge

### Train Accuracy Score: 
0.7471380471380472

### Test Accuracy Score: 
0.77183

At this point, I did not improve on my previous predictions. To approach a better score, I need to start feature engineering to produce a higher score. So far, Train accuracy scores appear below test accuracy scores. 

# Feature Engineering

### Datetime

In [ ]:
X = data
y = feature_info['status_group']

In [218]:
def dates(X):
    
    # Datetime Conversion
    X['date_recorded'] = pd.to_datetime(data['date_recorded'], infer_datetime_format = True)

    # Extract Year and Month
    X['Year'] = X['date_recorded'].map(lambda x: x.strftime('%Y'))
    X['Month'] = X['date_recorded'].map(lambda x: x.strftime('%m'))
    # Int Conversion
    X['Month_recorded'] = X['Month'].astype(int)
    X['Year_recorded'] = X['Year'].astype(int)
    
    # Seasonal Estiamtions
    # https://www.wildland.com/destinations/africa/tanzania/seasonsclimate.aspx
    # Deprecated due to lack of usefulness
    #hot_dry     = [ 12 , 1 , 2 ]
    #int_rains   = [ 3 ]
    #heavy_rains = [ 4 , 5 ]
    #cool_dry    = [ 6 , 7 , 8 , 9 , 10]
    #short_rain  = [ 11 ]
    
    # Feature Engineering
    #X['hot_dry'] = X['Month'].isin(hot_dry)
    #X['int_rains'] = X['Month'].isin(int_rains)
    #X['heavy_rains'] = X['Month'].isin(heavy_rains)
    #X['cool_dry'] = X['Month'].isin(cool_dry)
    #X['short_rain'] = X['Month'].isin(short_rain)
    
    # Drop Original Date Recorded Feature
    X = X.drop(columns = ['date_recorded'])
    
    return X

In [185]:
X = dates(X)
X_val = dates(X_val)

## Testing

In [186]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((47520, 46), (11880, 46), (47520,), (11880,))

In [187]:
pre_processor = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer (strategy = 'most_frequent')
)

X_train = pre_processor.fit_transform(X_train)
X_test = pre_processor.fit_transform(X_test)


In [188]:
param_distributions = {
    'n_estimators': randint(50, 500), 
    'max_depth': randint(1,5)
}

search = RandomizedSearchCV(
    estimator = XGBClassifier(n_jobs = -1, random_state = 42),
    param_distributions = param_distributions, 
    scoring = 'accuracy',
    n_iter = 50,
    n_jobs = -1,
    cv= 3,
    verbose = 10,
    return_train_score = True,
    random_state = 42
)

search.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   37.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 14.1min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 18.3min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 20.7min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 23.8min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
          fit_params=None, iid='warn', n_iter=50, n_jobs=-1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a419ff208>, 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a419ff668>},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=True, scoring='accuracy', verbose=10)

In [189]:
# creating variable name for the best_estimator object. 
best = search.best_estimator_
# Best Accuracy Score
search.best_score_

0.7375631313131313

In [190]:
best.fit(X_train, y_train)

permuter = PermutationImportance(best, scoring = 'accuracy',
                                 n_iter = 4, random_state = 42)

permuter.fit(X_test, y_test)

feature_names = X.columns.tolist()
eli5.show_weights(permuter, top = None, feature_names = feature_names)

Weight,Feature
0.0943 ± 0.0029,quantity
0.0281 ± 0.0050,waterpoint_type
0.0220 ± 0.0030,extraction_type_class
0.0128 ± 0.0017,construction_year
0.0118 ± 0.0017,payment
0.0061 ± 0.0009,longitude
0.0057 ± 0.0014,region
0.0051 ± 0.0009,amount_tsh
0.0039 ± 0.0009,source
0.0032 ± 0.0009,latitude


In [191]:
mask = permuter.feature_importances_ > 0
features = X.columns[mask]

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer (strategy = 'most_frequent'),
    best
)

pipeline.fit(X[features], y)

Pipeline(memory=None,
     steps=[('ordinalencoder', OrdinalEncoder(cols=['funder', 'installer', 'basin', 'region', 'lga', 'public_meeting', 'scheme_management', 'scheme_name', 'permit', 'extraction_type', 'extraction_type_class', 'management', 'payment', 'quality_group', 'quantity', 'source', 'source_type', 'source_class', '...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

In [192]:
cross_val_score(pipeline, X[features], y, scoring='accuracy', cv=3).mean()

0.7424579124579124

In [193]:
y_pred = pipeline.predict(X_val[features])
sample_submi['status_group'] = y_pred
sample_submi.to_csv("With_datetime_conversions.csv" , index = False)

In [194]:
!kaggle competitions submit ds3-predictive-modeling-challenge -f With_datetime_conversions.csv -m "Some feature engineering"

100%|█████████████████████████████████████████| 253k/253k [00:00<00:00, 293kB/s]
Successfully submitted to DS3 Predictive Modeling Challenge

### Train Accuracy Score: 
0.7424579124579124

### Test Accuracy Score: 
0.73756


## Improving low use features:

In [251]:
X = data
y = feature_info['status_group']

In [248]:
def encode(X):
    
    one_hot_features = ['quantity_group','quality_group','water_quality',
                        'extraction_type_group','management_group','extraction_type']
    one_hot = ce.OneHotEncoder(use_cat_names=True)
    oh_features = one_hot.fit_transform(X[one_hot_features]).columns

    X[oh_features] = one_hot.fit_transform(X[one_hot_features])
    
    X = X.drop(columns = one_hot_features)
    
    return X



In [243]:
"""one_hot_features = ['quantity_group','quality_group','water_quality',
                        'extraction_type_group','management_group','extraction_type']
one_hot = ce.OneHotEncoder(use_cat_names=True)
oh_features = one_hot.fit_transform(X[one_hot_features]).columns

X[oh_features] = one_hot.fit_transform(X[one_hot_features])"""

"one_hot_features = ['quantity_group','quality_group','water_quality',\n                        'extraction_type_group','management_group','extraction_type']\none_hot = ce.OneHotEncoder(use_cat_names=True)\noh_features = one_hot.fit_transform(X[one_hot_features]).columns\n\nX[oh_features] = one_hot.fit_transform(X[one_hot_features])"

In [256]:
X.shape

(59400, 92)

In [252]:
X = dates(X)
X_val = dates(X_val)

X = encode(X)
X_val = encode(X_val)

In [258]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((47520, 92), (11880, 92), (47520,), (11880,))

In [259]:
pre_processor = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer (strategy = 'most_frequent')
)

X_train = pre_processor.fit_transform(X_train)
X_test = pre_processor.fit_transform(X_test)


In [261]:
param_distributions = {
    'n_estimators': randint(50, 500), 
    'max_depth': randint(1,5)
}

search = RandomizedSearchCV(
    estimator = XGBClassifier(n_jobs = -1, random_state = 42),
    param_distributions = param_distributions, 
    scoring = 'accuracy',
    n_iter = 3,
    n_jobs = -1,
    cv= 3,
    verbose = 10,
    return_train_score = True,
    random_state = 42
)

search.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed:  1.2min remaining:  4.1min
[Parallel(n_jobs=-1)]: Done   3 out of   9 | elapsed:  1.2min remaining:  2.4min
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:  1.2min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done   5 out of   9 | elapsed:  1.2min remaining:   58.3s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:  1.8min remaining:   53.9s
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:  2.8min remaining:   47.9s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  2.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  2.9min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
          fit_params=None, iid='warn', n_iter=3, n_jobs=-1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a41fd6eb8>, 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a41fd6198>},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=True, scoring='accuracy', verbose=10)

In [262]:
# creating variable name for the best_estimator object. 
best = search.best_estimator_
# Best Accuracy Score
search.best_score_

0.7290824915824916

In [263]:
best.fit(X_train, y_train)

permuter = PermutationImportance(best, scoring = 'accuracy',
                                 n_iter = 4, random_state = 42)

permuter.fit(X_test, y_test)

feature_names = X.columns.tolist()
eli5.show_weights(permuter, top = None, feature_names = feature_names)

Weight,Feature
0.0381 ± 0.0018,quantity
0.0184 ± 0.0037,quantity_group_dry
0.0160 ± 0.0006,waterpoint_type
0.0111 ± 0.0011,construction_year
0.0076 ± 0.0012,payment
0.0069 ± 0.0007,extraction_type_class
0.0049 ± 0.0008,longitude
0.0044 ± 0.0036,region
0.0041 ± 0.0028,extraction_type_group_other
0.0034 ± 0.0014,lga


In [264]:
mask = permuter.feature_importances_ > 0
features = X.columns[mask]

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer (strategy = 'most_frequent'),
    best
)

pipeline.fit(X[features], y)

Pipeline(memory=None,
     steps=[('ordinalencoder', OrdinalEncoder(cols=['funder', 'installer', 'wpt_name', 'basin', 'region', 'lga', 'ward', 'public_meeting', 'scheme_management', 'extraction_type_class', 'management', 'payment', 'quantity', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group'...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

In [265]:
cross_val_score(pipeline, X[features], y, scoring='accuracy', cv=3).mean()

0.7430976430976431

In [266]:
y_pred = pipeline.predict(X_val[features])
sample_submi['status_group'] = y_pred
sample_submi.to_csv("With_One_Hot.csv" , index = False)
!kaggle competitions submit ds3-predictive-modeling-challenge -f With_One_Hot.csv -m "Now with One-Hot"

100%|█████████████████████████████████████████| 254k/254k [00:00<00:00, 305kB/s]
Successfully submitted to DS3 Predictive Modeling Challenge

In [272]:
X = X[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((47520, 40), (11880, 40), (47520,), (11880,))

In [273]:
pre_processor = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer (strategy = 'most_frequent')
)

X_train = pre_processor.fit_transform(X_train)
X_test = pre_processor.fit_transform(X_test)


In [274]:
param_distributions = {
    'n_estimators': randint(50, 500), 
    'max_depth': randint(1,5)
}

search = RandomizedSearchCV(
    estimator = XGBClassifier(n_jobs = -1, random_state = 42),
    param_distributions = param_distributions, 
    scoring = 'accuracy',
    n_iter = 20,
    n_jobs = -1,
    cv= 3,
    verbose = 10,
    return_train_score = True,
    random_state = 42
)

search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done  52 out of  60 | elapsed:  9.2min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 10.5min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
          fit_params=None, iid='warn', n_iter=20, n_jobs=-1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a33539748>, 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a33539860>},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=True, scoring='accuracy', verbose=10)

In [275]:
# creating variable name for the best_estimator object. 
best = search.best_estimator_
# Best Accuracy Score
search.best_score_

0.7261153198653199

In [276]:
best.fit(X_train, y_train)

permuter = PermutationImportance(best, scoring = 'accuracy',
                                 n_iter = 4, random_state = 42)

permuter.fit(X_test, y_test)

feature_names = X.columns.tolist()
eli5.show_weights(permuter, top = None, feature_names = feature_names)

Weight,Feature
0.0386 ± 0.0029,quantity
0.0166 ± 0.0011,quantity_group_dry
0.0159 ± 0.0044,waterpoint_type
0.0090 ± 0.0012,construction_year
0.0084 ± 0.0017,payment
0.0052 ± 0.0028,extraction_type_class
0.0043 ± 0.0013,extraction_type_group_other
0.0041 ± 0.0011,longitude
0.0036 ± 0.0026,region
0.0031 ± 0.0021,source


In [277]:
mask = permuter.feature_importances_ > 0
features = X.columns[mask]

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer (strategy = 'most_frequent'),
    best
)

pipeline.fit(X[features], y)

Pipeline(memory=None,
     steps=[('ordinalencoder', OrdinalEncoder(cols=['funder', 'installer', 'wpt_name', 'basin', 'region', 'lga', 'public_meeting', 'scheme_management', 'extraction_type_class', 'payment', 'quantity', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group'],
        drop_invari...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

In [278]:
cross_val_score(pipeline, X[features], y, scoring='accuracy', cv=3).mean()

0.7421717171717171

### Trying Random Forest Instead

In [297]:
X = data
y = feature_info['status_group']

X = dates(X)
X_val = dates(X_val)

X = encode(X)
X_val = encode(X_val)

In [298]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((47520, 92), (11880, 92), (47520,), (11880,))

In [299]:
pre_processor = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer (strategy = 'most_frequent')
)

X_train = pre_processor.fit_transform(X_train)
X_test = pre_processor.fit_transform(X_test)

In [300]:
from sklearn.ensemble import RandomForestClassifier

param_distributions = {
    'n_estimators': randint(50, 500), 
    'max_depth': randint(2,5)
}

search = RandomizedSearchCV(
    estimator = RandomForestClassifier(n_jobs = -1, random_state = 42),
    param_distributions = param_distributions, 
    scoring = 'accuracy',
    n_iter = 20,
    n_jobs = -1,
    cv= 3,
    verbose = 10,
    return_train_score = True,
    random_state = 42
)

search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   37.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   51.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  52 out of  60 | elapsed:  1.6min remaining:   14.5s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.7min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=20, n_jobs=-1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a540ff470>, 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a587b5fd0>},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=True, scoring='accuracy', verbose=10)

In [301]:
# creating variable name for the best_estimator object. 
best = search.best_estimator_
# Best Accuracy Score
search.best_score_

0.7103114478114478

In [302]:
best.fit(X_train, y_train)

permuter = PermutationImportance(best, scoring = 'accuracy',
                                 n_iter = 4, random_state = 42)

permuter.fit(X_test, y_test)

feature_names = X.columns.tolist()
eli5.show_weights(permuter, top = None, feature_names = feature_names)

Weight,Feature
0.0153 ± 0.0027,quantity
0.0100 ± 0.0027,quantity_group_dry
0.0091 ± 0.0015,waterpoint_type
0.0058 ± 0.0012,waterpoint_type_group
0.0027 ± 0.0015,construction_year
0.0027 ± 0.0006,extraction_type_class
0.0025 ± 0.0006,region
0.0024 ± 0.0010,lga
0.0023 ± 0.0004,amount_tsh
0.0014 ± 0.0007,extraction_type_group_other


In [303]:
mask = permuter.feature_importances_ > 0
features = X.columns[mask]

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer (strategy = 'most_frequent'),
    best
)

pipeline.fit(X[features], y)

cross_val_score(pipeline, X[features], y, scoring='accuracy', cv=3).mean()

0.6994444444444445